# Generating Powersets: Two Approaches

In this post, we will generate the powerset of a given set using two equivalent algorithms.  The powerset of a set is the collection of all possible subsets of the original set, including the empty set and itself as members. In order include subsets as members of the powerset, we will define a new object class, `ModSet()`, to encapsulate member sets so that they are "hashable".  Below, we'll explain what Python means by this.

Now, a bit about powersets.  

If the source set has $n$ members, its powerset will have $2^n$ subsets when including the source set itself and the empty set as members. This follows from the theorem that all possible subsets can be generated by selecting members by digit occurance in the base-2 numbers counting from 0 to n.  This direct approach has a certain efficiency and elegance; I might go so far to say that it is the best way to generate a powerset. We will call this method "The Binary Mask approach".

But there is another approach you can use...

Alternatively, you can take your source set, remove one of its members, then define a new subset with the members that remain. Repeating this process, using all elements on all subsets, you will produce the powerset of the source set as well.  This second approach we call here, "The Recursive Approach".  While this latter approach may lack the directness of the binary mask, we hope that coding it will prove to be a worthwhile exercise in recursion.

## The Python set class 
The set class in python takes only immutable types (i.e. strings, numeric values, and tuples) as members. When you try to insert a mutable type, lists or dictionaries for example, python will spit back an 'unhashable' type error.  

In [1]:
a = set(['covefefe', 1.6, 2021, (45,'MAGA')])
a.add('RINO')
a

{(45, 'MAGA'), 1.6, 2021, 'RINO', 'covefefe'}

In [2]:
a.add(['26th','ammendment'])

TypeError: unhashable type: 'list'

Because sets are mutable, the same goes if you attempt to include a set within another set.

In [3]:
b = set(['impeachment', 'procedings'])
b

{'impeachment', 'procedings'}

In [4]:
a.add(b)

TypeError: unhashable type: 'set'

Though you can join sets together, by combining unique members into a single pool, with the `set.union()` operation.

In [5]:
a = a.union(b)
a

{(45, 'MAGA'), 1.6, 2021, 'RINO', 'covefefe', 'impeachment', 'procedings'}

Now, because separate instances of a class are allocated their own locations in memory, they are hashable; even if they contain unhashable values as attributes. And, even if the values of the attributes of the two instances are equivalent.

In [6]:
class Advisor():
    def __init__(self):
        self.val = {'Presidential', 'pardon'}  # Attribute contains unhashable set
        
RogerS = Advisor()   # has set for attribute .val
PaulM = Advisor()  # has same set as above for its attribute .val
c = set([PaulM, RogerS])  # include the two instances as members of a set
print(type(c))           # c is indeed a set
d = [type(el.val) for el in c]  # attributes of member instances are sets
d

<class 'set'>


[set, set]

We can also do one better and instruct that instances of the class, when called, define themselves as their `.val` attribute.

In [73]:
class Advisor():
    def __init__(self):
        self.val = {'Presidential', 'pardon'}
    def __repr__(self):
        return self.val.__repr__()

GeorgeF = Advisor()
e = set([GeorgeF])
e

{{'pardon', 'Presidential'}}

But don't be fooled. While we have the *appearence* of having a set contained  within a set, what we actually have is a set containing an instance of `Advisor()` having a set as an attribute.

In [77]:
print(type(e))
print(type(list(e)[0]))
print(type(list(e)[0].val))

<class 'set'>
<class '__main__.Advisor'>
<class 'set'>


In code that follows, we'll define a class with the same properties to include subsets in an all-encompassing powerset. More than a cosmetic device, defining the set container in the above manner will allow powersets to be generated using a recursion-based algorthm.

## Define a set container class

Just like in the illustrative example above, we will define a new class, `ModSet()`, with the same constructor and representative methods. New instances require a set argument, either occupied or empty. In addition, we will define methods for copying and nesting operations.

In [10]:
class ModSet():
    
    # Instance constructor
    def __init__(self, setElement):
        self.val = setElement  # set arg saved in .val attribute
    
    # String eval representation of self instance 
    def __repr__(self):
        return self.val.__repr__() # Return string eval represent.
                                   # of string object in .val
    
    # Method to make a copy of self instance
    def __copy__(self):
        return ModSet(self.val)
    
    # Modify .val to contain the set of itself, of itself, ...
    # nesting .val "depth" number of levels.
    def pushDown(self, depth):
        while depth > 0:
            self.val = set([ModSet(self.val)])
            depth -= 1     
    
    # Remove one nesting level from set in self.val.
    # If un-nested, ignore.
    def pullUpOneLevel(self):
        listSet = list(self.val)
        if len(listSet) == 1:
            self.val = listSet[0].val
        else:
            pass
    
    # Remove height number of nesting levels from set in
    # self.val by repeatedly calling above method
    def pullUp(self, height):
        while height > 0:
            self.pullUpOneLevel()
            height -= 1
                

Testing these, let's define a set of mixed immutables and nest it 3 levels deep.

In [21]:
prez = ModSet(set(['covefefe', ('impeachment', '#2'), 1.6, 2021]))
prez.pushDown(3)
prez

{{{{1.6, 'covefefe', 2021, ('impeachment', '#2')}}}}

Raising that result back up three levels should return the set to how it was originally defined.

In [22]:
prez.pullUp(3)
prez

{1.6, 'covefefe', 2021, ('impeachment', '#2')}

If you instruct to pull up greater than the set is deep, the extra assentions are ignored.

In [23]:
prez.pullUp(2)
prez

{1.6, 'covefefe', 2021, ('impeachment', '#2')}

### Uniqueness

Just like Python's set class, we will make sure that each member of ModSet be unique according to the value of its `.val` attribute.  When a ModSet is comprised of immutables, the set class automatically takes care of uniqueness for us.  However, if a ModSet contains other ModSets, we require an additional method to enforce uniqueness among them as well.

In [ ]:
class ModSet():
    .
    .
    .
    def removeDuplicates(self):
        uniqueSet = set()
        for s in self.val:
            inUniqueSet = False
            for us in uniqueSet:
                if us.val == s.val:
                    inUniqueSet = True
            if not inUniqueSet:
                uniqueSet.add(s)
        self.val = uniqueSet

After including `removeDuplicates()` in our class definition we'll test it as follows. 

In [195]:
prez1 = ModSet(set(['covefefe', ('impeachment', '#2'), 1.6, 2021]))  # ModSet instance no. 1
prez2 = ModSet(set(['covefefe', ('impeachment', '#2'), 1.6, 2021]))  # ModSet instance no. 2
prez3 = ModSet(set(['covefefe', ('impeachment', '#2'), 1.6, 2021]))  # ModSet instance no. 3
print(set([id(prez1), id(prez2), id(prez3)]))  # each instance held in its own memory block

modSetOfPrez = ModSet(set([prez1, prez2, prez3]))  # define a ModSet of ModSet instances
print(modSetOfPrez)  # set class sees each instance as unique though they contain same values
                     # in their attributes

modSetOfPrez.removeDuplicates()  # enforce uniqueness according to attribute value
print(modSetOfPrez)   # result after applying removeDuplicates()

{140055637076848, 140055637077072, 140055637076400}
{{1.6, 'covefefe', 2021, ('impeachment', '#2')}, {1.6, 'covefefe', 2021, ('impeachment', '#2')}, {1.6, 'covefefe', 2021, ('impeachment', '#2')}}
{{1.6, 'covefefe', 2021, ('impeachment', '#2')}}


### Set operator methods

For the powerset routines to come, we need ModSets to join by union and also separate by set-subtraction.  Below we define these as well as intersection.

In [ ]:
class ModSet():
    .
    .
    .
    # union join multiple items, enforce that ModSet members are unique
    def union(self, *modSets):
        for modSet in modSets:
            self.val = self.val.union(modSet.val)  # this is union method from set class
            self.removeDuplicates()  # removes duplicate-valued instances of ModSet members
    
    # set intersect multiple items
    def intersection(self, *modSets):
        for modSet in modSets:
            self.val = self.val.intersection(modSet.val)
    
    # set difference multiple items. Note: arg order matters here! 
    def difference(self, *modSets):
        for modSet in modSets:
            self.val = self.val.difference(modSet.val)
    
    # version of above method that returns a new instance for assignment.
    # Note: only two arguments here.
    def diffFunc(modSet1, modSet2):
        return ModSet(set.difference(modSet1.val, modSet2.val))

Test union joining operation of non-ModSet objects

In [198]:
charges = ModSet(set())
charge1 = ModSet(set(['obstruction', 'of', 'justice']))
charge2 = ModSet(set(['abuse', 'of', 'power']))
charge3 = ModSet(set(['inciting', 'insurrection']))

charges.union(charge1, charge2, charge3)
print(charges)

{'inciting', 'insurrection', 'of', 'abuse', 'justice', 'power', 'obstruction'}


The three separate ModSets have been merged into a single ModSet, `charges`.  Notice that string member `'of'` appears only once, as it should, though it is present in both `charge1` and `charge2`.  Now, we'll test union operation with ModSet members as well.

In [199]:
charge1.pushDown(1)
charge2.pushDown(1)
charge3.pushDown(1)

charges.union(charge1, charge1, charge2, charge3, ModSet(set(['alternative', 'facts'])))
print(charges)

{{'insurrection', 'inciting'}, {'justice', 'obstruction', 'of'}, 'inciting', 'insurrection', 'of', 'alternative', 'abuse', 'obstruction', 'justice', 'power', {'power', 'abuse', 'of'}, 'facts'}


In [194]:
chargesSubset1 = ModSet(set(['rounding', 'the', 'bend']))
chargesSubset2 = ModSet(set(['thwarting', 'the', 'dems']))
chargesSubset1.union(charge1, charge2)
chargesSubset2.union(charge2, charge3)
chargesSubset1.intersection(chargesSubset2)
chargesSubset1

{{{'power', 'abuse', 'of'}}, 'the'}

In [189]:
charges.difference(charge2)
charges

{'inciting', 'insurrection', {'justice', 'obstruction', 'of'}, 'of', 'alternative', {'insurrection', 'inciting'}, 'abuse', 'obstruction', 'justice', 'power', 'facts'}

In [190]:
chargesReduced = ModSet.diffFunc(charges, charge1)
chargesReduced

{'facts', 'inciting', 'insurrection', 'of', 'alternative', {'insurrection', 'inciting'}, 'abuse', 'justice', 'power', 'obstruction'}

In [197]:
class ModSet():
    
    # Instance constructor
    def __init__(self, setElement):
        self.val = setElement  # set arg saved in .val attribute
    
    # String eval representation of self instance 
    def __repr__(self):
        return self.val.__repr__() # Return string eval represent.
                                   # of string object in .val
    
    # Method to make a copy of self instance
    def __copy__(self):
        return ModSet(self.val)
    
    # Modify .val to contain the set of itself, of itself, ...
    # nesting .val "depth" number of levels.
    def pushDown(self, depth):
        while depth > 0:
            self.val = set([ModSet(self.val)])
            depth -= 1     
    
    # Remove one nesting level from set in self.val.
    # If un-nested, ignore.
    def pullUpOneLevel(self):
        listSet = list(self.val)
        if len(listSet) == 1:
            self.val = listSet[0].val
        else:
            pass
    
    # Remove "height" nesting levels from set in
    # self.val by repeatedly calling above method
    def pullUp(self, height):
        while height > 0:
            self.pullUpOneLevel()
            height -= 1
    
    # Within a single set, multiple ModSets with 
    # equivalent .val attributes can exist.  
    # This can occur because each instance of ModSet
    # allocated a unique location in memory. For sets
    # to be unique in terms of ModSet.val values we
    # define the following filtering method.
    #def removeDuplicates(self):
    #    uniqueSet = set()
    #    for s in self.val:
    #        inUniqueSet = False
    #        for us in uniqueSet:
    #            if us.val == s.val:
    #                inUniqueSet = True
    #        if not inUniqueSet:
    #            uniqueSet.add(s)
    #    self.val = uniqueSet 
        
    def removeDuplicates(self):
        uniqueSet = set()
        for s in self.val:  # s is a member of the ModSet().val set
            inUniqueSet = False  # match detetion flag to false
            sTest = s
            for us in uniqueSet: # us is a member of the uniqueSet set
                usTest = us
                if isinstance(us, ModSet): # if element us is a ModSet
                    usTest = us.val        # change test value to its attribute
                if isinstance(s, ModSet):  # if element s is a ModSet
                    sTest = s.val          # change test value to its attribute
                if usTest == sTest:  # determine if values match on this run
                    inUniqueSet = True  # if so, set the flag to true
            if not inUniqueSet:    # only add member s to uniqueSet if
                uniqueSet.add(s)   # match is NOT detected
        self.val = uniqueSet
    
    # union join multiple items, enforce that ModSet members are unique
    def union(self, *modSets):
        for modSet in modSets:
            self.val = self.val.union(modSet.val)  # this is union method from set class
            self.removeDuplicates()  # removes duplicate-valued instances of ModSet members
    
    # set intersect multiple items
    def intersection(self, *modSets):
        for modSet in modSets:
            self.val = self.val.intersection(modSet.val)
    
    # set difference multiple items. note: arg order matters here! 
    def difference(self, *modSets):
        for modSet in modSets:
            self.val = self.val.difference(modSet.val)
    
    # version of above method that returns a new instance,
    # only takes two arguments.
    def diffFunc(modSet1, modSet2):
        return ModSet(set.difference(modSet1.val, modSet2.val))
    
    # Generate powerset via binary mask assignment of set members
    def powerSet_bin(self):
        S = list(self.val)         # convert to list for indexing
        setSize = len(self.val)    # count number of members in set
        psetSize = pow(2, setSize) # calc number of elements in powerset
        lastIndex = setSize - 1    # index value of last member
        #bMask = [[False]*setSize]*psetSize  # Initialize binary mask
        bMask = np.zeros([psetSize, setSize]).astype(bool)
        setIndices = range(0,setSize)
        psetIndices = range(0,psetSize)
        pSet = ModSet(set([]))
        pSet.pushDown(1) 
        for i in psetIndices:
            diff = i
            for j in setIndices:
                if (diff >= pow(2, lastIndex - j)): 
                    bMask[i, lastIndex - j] = True
                    diff -= pow(2, lastIndex - j)
            # Write current subset to power set list. Using list
            # comprehension here for compactness.
            dummySet = ModSet(set([S[k] for k in setIndices if bMask[i,k] == True]))
            dummySet.pushDown(1)
            pSet.union(dummySet)
        return pSet, bMask
    
    # Generate powerset recursively.
    def powerSet_rec(self):
        
        pSet = self.__copy__()      # Preserve self, its copy will be altered
        pSet.pushDown(1)            #
              
        if len(self.val) > 0:       # Recursion termination condition
            for elSet in self.val:  # Iterate through members of set self.val
                # Generate subset that remains after removing current
                # element, elSet, from set self.val
                dummySet = self.diffFunc(ModSet(set([elSet]))) 
                # To current powerset, append the powerset of the
                # subset from previous step
                pSet.union(dummySet.powerSet_rec())  # Self-call powerset method,
                                                     # union join powerset of 
                                                     # dummySet with pSet
            return pSet             # Return powerset at current 
                                    # level of recursion
        else:
            dummySet = ModSet(set())  # Generate instance of ModSet of empty set
            dummySet.pushDown(1)      # Nest empty set down one level so it can
            return dummySet           # be union joined in the recursion level
                                      # above that called this run-through.